In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [9]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
params = pd.read_csv(path+'parameters/random-forest-regression.csv')

In [10]:
### Season to test results

N = 2021

In [29]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(['driver_points_after_race'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_after_race.values)

In [24]:
def score_regression(model):
    correct_predictions = 0
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        # X_test = test.drop(['podium'], axis=1)
        X_test = test.drop(['driver_points_after_race'], axis=1)
        y_test = test.driver_points_after_race

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['driver_points_after_race'] = y_test.reset_index(drop=True)
        # prediction_df['podium'] = y_test.reset_index(drop=True)
        prediction_df['actual'] = prediction_df.driver_points_after_race
        prediction_df.sort_values('results', ascending=False, inplace=True)
        prediction_df.reset_index(inplace=True, drop=True)
        prediction_df['predicted'] = prediction_df.index

        correct_predictions += precision_score(prediction_df.actual, prediction_df.predicted)

    return correct_predictions / df[df.season == N]['round'].nunique()

In [25]:
# Random Forest Regressor
p = params.head(1)
c = p.criterion[0]
max_f = p.max_features[0]
max_d = p.max_depth[0]

model_params = (c, max_f, max_d)
model = RandomForestRegressor(criterion=c, max_features=max_f, max_depth=max_d)
model.fit(X_train, y_train)

score = score_regression(model)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [23]:
score

0.0